In [1]:
from os import listdir, system, makedirs
from os.path import isfile, join, exists

from scipy.stats import skew, kurtosis
import numpy as np
import pickle

import parselmouth as pm
from tysums import *
from henrys import *

clip_length = 5000
clip_period = 500

PATH_TO_MTC_DATA = '/srv/mtc_data'
PATH_TO_AUDIO = join(PATH_TO_MTC_DATA, 'tpm-f2020-project/mtc-audio-TRC')
PATH_TO_SPLIT_AUDIO = join(PATH_TO_MTC_DATA, 'tpm-f2020-project/mtc-split-wav-files')

SPLIT_AUDIO_DIR_CONVENTION = 'length-{}-period-{}'
DATA_POINT_DIR_CONVENTION = 'starttime-{}-title-{}'
SPLIT_AUDIO_FILE_CONVENTION = 'last-{}.wav'
QUESTION_TRUTH_LABELS_CONVENTION = 'tpm-f2020-project/length-{}-period-{}-question_truth_labels.pkl'

In [2]:
f = open(join(PATH_TO_MTC_DATA, 'tpm-f2020-project/batches_dirs.pkl'), "rb")
batches_dirs = pickle.load(f)
f.close()

f = open(join(PATH_TO_MTC_DATA, 'tpm-f2020-project/batches_labels.pkl'), "rb")
batches_labels = pickle.load(f)
f.close()

In [8]:
data_matrix = []
labels_array = []

In [26]:
for i in range(30, 40):
    print(i)
    batch_dirs = batches_dirs[i]
    batch_labels = batches_labels[i]

    for j in range(len(batch_dirs)):
        data_point_dir = batch_dirs[j]
        label = batch_labels[j]
        try:
            full_audio = pm.Sound(join(data_point_dir, SPLIT_AUDIO_FILE_CONVENTION.format(0)))

            t_end = full_audio.get_end_time()
            t_beg_last_500 = t_end - 0.5
            t_beg_last_500 = t_beg_last_500 if t_beg_last_500 > 0 else 0
            t_beg_last_200 = t_end - 0.2
            t_beg_last_200 = t_beg_last_200 if t_beg_last_200 > 0 else 0

            last_500 = full_audio.extract_part(from_time=t_beg_last_500, to_time=t_end)
            last_200 = full_audio.extract_part(from_time=t_beg_last_200, to_time=t_end)

            audio_clips = [full_audio, last_500, last_200]
            feature_vals = []

            for audio_clip in audio_clips:
                sample_freq = audio_clip.sampling_frequency

                f0 = calculate_fundemental_frequency(audio_clip)
                feature_vals.extend(get_stats(f0, sample_freq))

                signal_intensity = calculate_signal_intensity(audio_clip)[:,0]
                feature_vals.extend(get_stats(signal_intensity, sample_freq))

                jitter = calculate_jitter_of_audio_clip(audio_clip)
                feature_vals.append(jitter if not np.isnan(jitter) else 0)

                shimmer = calculate_shimmer_of_audio_clip(audio_clip)
                feature_vals.append(shimmer if not np.isnan(shimmer) else 0)

                mfcc_coeff_vectors = calculate_MFCC_coefficients(audio_clip)
                for mfcc_coeff_vector in mfcc_coeff_vectors:
                    feature_vals.extend([np.mean(mfcc_coeff_vector), np.std(mfcc_coeff_vector)])

                rms = calculate_RMS_frame_energy(audio_clip)
                feature_vals.append(rms)

            num_pauses, pause_durations, pause_starts, pause_total_length = calculate_pauses(full_audio)
            feature_vals.extend([num_pauses, pause_total_length])
            feature_vals.extend(get_pause_stats(pause_durations, pause_starts))

            zcr = calculate_zero_crossing_rate(full_audio)
            feature_vals.append(zcr)

            spectral_balance = calculate_spectral_balance(full_audio)
            feature_vals.append(spectral_balance if not np.isnan(spectral_balance) else 0)

            average_psd = calculate_power_spectral_density(full_audio)
            feature_vals.append(np.argmax(average_psd))
            feature_vals.append(np.max(average_psd))

            data_matrix.append(np.array(feature_vals))
            labels_array.append(label)
        except Exception as e:
            print(e.with_traceback())
            
f = open(join(PATH_TO_MTC_DATA, 'tpm-f2020-project/data_matrix.pkl'), "wb")
pickle.dump(data_matrix, f)
f.close()

f = open(join(PATH_TO_MTC_DATA, 'tpm-f2020-project/labels_array.pkl'), "wb")
pickle.dump(labels_array, f)
f.close()

30
31
32
33
34
35
36
37
38
39


In [28]:
len(labels_array)

20000

In [17]:
f = open(join(PATH_TO_MTC_DATA, 'tpm-f2020-project/data_matrix.pkl'), "rb")
test_matrix = pickle.load(f)
f.close()

f = open(join(PATH_TO_MTC_DATA, 'tpm-f2020-project/labels_array.pkl'), "rb")
test_array = pickle.load(f)
f.close()

In [21]:
len(data_matrix)

5000

In [18]:
full_data_matrix = np.array(test_matrix)
full_labels_array = np.array(test_array)

In [20]:
full_data_matrix.shape
full_labels_array.shape

(5000, 194)